In [22]:
# # Colab only
# %%javascript
# function ClickConnect(){
# console.log("Working");
# document.querySelector("colab-toolbar-button#connect").click()
# }setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [23]:
# # Check if GPU is available
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 472850192964110051
]


In [24]:
# # Colab only
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Questão 1

## 1.1. Import da MobileNetV2

In [25]:
from keras.applications import mobilenet_v2

In [26]:
mobile = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
# mobile.summary()

## 1.2. Congelar parâmetros

In [27]:
for layer in mobile.layers:
        layer.trainable = False

## 1.3. Uma camada densa com ativação softmax

In [28]:
from keras.models import Model
from keras.layers import Dense, Flatten

In [29]:
out_new_model = mobile.output
out_new_model = Flatten()(out_new_model)
out_new_model = Dense(7, activation='softmax')(out_new_model)

new_model = Model(inputs=mobile.input, outputs=out_new_model)
# new_model.summary()

## 1.4. Configurando processo de otimização

In [30]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [31]:
print("weights:", len(new_model.weights))
print("trainable_weights:", len(new_model.trainable_weights))
print("non_trainable_weights:", len(new_model.non_trainable_weights))

weights: 262
trainable_weights: 2
non_trainable_weights: 260


# 2. Questão 2

In [32]:
# fix from: https://github.com/keras-team/keras/issues/9624

def preprocess_input_new(x):
    img = mobilenet_v2.preprocess_input(image.img_to_array(x))
    return image.array_to_img(img)

## 2.1. Data Augmentation

In [33]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [34]:
batch_size = 32
data_dir = './data/dataset/'
# data_dir = '/content/drive/MyDrive/usp/7 semestre/redes neurais/trab3/data/dataset/'
n_epochs = 10

datagen = image.ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True, 
    height_shift_range=0.1,
    width_shift_range=0.1,
    preprocessing_function=preprocess_input_new)

train_generator = datagen.flow_from_directory(
    data_dir + 'train',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    data_dir + 'test',
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical')

Found 2284 images belonging to 7 classes.
Found 1400 images belonging to 7 classes.


## 2.2. Fit

In [35]:
new_model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = n_epochs)

Epoch 1/10
71/71 [==============================] - 70s 854ms/step - loss: 3.5344 - accuracy: 0.3203 - val_loss: 1.6173 - val_accuracy: 0.5385
Epoch 2/10
71/71 [==============================] - 56s 794ms/step - loss: 1.3362 - accuracy: 0.6064 - val_loss: 1.5064 - val_accuracy: 0.5610
Epoch 3/10
71/71 [==============================] - 57s 798ms/step - loss: 1.0388 - accuracy: 0.6651 - val_loss: 1.2151 - val_accuracy: 0.6344
Epoch 4/10
71/71 [==============================] - 57s 804ms/step - loss: 0.9025 - accuracy: 0.7049 - val_loss: 1.1829 - val_accuracy: 0.6366
Epoch 5/10
71/71 [==============================] - 57s 807ms/step - loss: 0.8684 - accuracy: 0.7259 - val_loss: 1.2981 - val_accuracy: 0.6214
Epoch 6/10
71/71 [==============================] - 57s 807ms/step - loss: 0.8136 - accuracy: 0.7405 - val_loss: 1.3622 - val_accuracy: 0.6642
Epoch 7/10
71/71 [==============================] - 56s 789ms/step - loss: 0.9073 - accuracy: 0.7311 - val_loss: 2.3422 - val_accuracy: 0.5349

## 2.3. Teste

In [36]:
import numpy as np

In [37]:
# img = image.load_img('./data/dataset/test/bart_simpson/pic_0062.jpg', 
#                       target_size=(128, 128))
img = image.load_img('/content/drive/MyDrive/usp/7 semestre/redes neurais/trab3/data/dataset/test/bart_simpson/pic_0062.jpg', 
                      target_size=(128, 128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = mobilenet_v2.preprocess_input(x) 

In [38]:
preds = new_model.predict(x)

print(preds.max(), preds.argmax())

0.92361504 1
